# Project Progress

## Initial Imports

In [17]:
import sys
from pathlib import Path

root_dir = Path().absolute().parent.parent
sys.path.append(str(root_dir))

In [18]:
# Warning control
import json
import os
import warnings
import yaml
from dotenv import load_dotenv

from crewai import Agent, Task, Crew

warnings.filterwarnings('ignore')

load_dotenv()

True

## Load OpenAI Model

In [19]:
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'

## Loading Tasks and Agents YAML files

In [20]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

## Creating Custom Tools

In [37]:
from typing import Type

import asana
from crewai_tools import BaseTool
from pydantic import BaseModel, Field


class ProjectTasksFetcherTool(BaseTool):
    name: str = "Asana Project Tasks Fetcher"
    description: str = "Fetches all tasks and their details from an Asana project"
    
    def _run(self) -> dict:
        """
        Fetch all tasks from the specified Asana project.
        """
        configuration = asana.Configuration()
        configuration.access_token = os.environ['ASANA_ACCESS_TOKEN']
        api_client = asana.ApiClient(configuration)
        
        tasks_api = asana.TasksApi(api_client)
        project_gid = os.environ['ASANA_PROJECT_GID']
        
        opts = {
            'completed_since': 'now',
            'limit': 100,
            'opt_fields': (
                "gid,name,notes,completed,completed_at,due_on,due_at,"
                "assignee.name,tags.name,memberships.section.name"
            )
        }
        
        try:
            tasks = tasks_api.get_tasks_for_project(project_gid, opts)
            return [task for task in tasks]
        except asana.rest.ApiException as e:
            return {"error": f"Failed to fetch project tasks: {str(e)}"}

class TaskInput(BaseModel):
        """Input schema for TaskDataFetcherTool."""
        task_gid: str = Field(..., description="Asana task GID to fetch details for")

class TaskDataFetcherTool(BaseTool):
    name: str = "Asana Task Data Fetcher"
    description: str = "Fetches detailed data for a specific Asana task. Requires task_gid as input."
    args_schema: Type[BaseModel] = TaskInput  # We still need this for the task_gid input

    def _run(self, task_gid: str) -> dict:
        """
        Fetch detailed data for a specific task.
        """
        configuration = asana.Configuration()
        configuration.access_token = os.environ['ASANA_ACCESS_TOKEN']
        api_client = asana.ApiClient(configuration)
        
        tasks_api = asana.TasksApi(api_client)
        
        opts = {
            'opt_fields': (
                "name,notes,completed,completed_at,due_on,due_at,"
                "assignee.name,tags.name,memberships.section.name,"
                "html_notes,followers.name,num_subtasks"
            )
        }
        
        try:
            task = tasks_api.get_task(task_gid, opts)
            return task
        except asana.rest.ApiException as e:
            return {"error": f"Failed to fetch task data: {str(e)}"}

## Create Crew, Agents and Tasks

In [38]:
access_token = os.environ['ASANA_ACCESS_TOKEN']
project_gid = os.environ['ASANA_PROJECT_GID']

# Creating Agents
data_collection_agent = Agent(
  config=agents_config['data_collection_agent'],
  tools=[
    ProjectTasksFetcherTool(),
    TaskDataFetcherTool()
  ]
)

analysis_agent = Agent(
  config=agents_config['analysis_agent']
)

# Creating Tasks
data_collection = Task(
  config=tasks_config['data_collection'],
  agent=data_collection_agent
)

data_analysis = Task(
  config=tasks_config['data_analysis'],
  agent=analysis_agent
)

report_generation = Task(
  config=tasks_config['report_generation'],
  agent=analysis_agent,
)

# Creating Crew
crew = Crew(
  agents=[
    data_collection_agent,
    analysis_agent
  ],
  tasks=[
    data_collection,
    data_analysis,
    report_generation
  ],
  verbose=True
)

2024-12-16 17:44:56,818 - 11692 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


## Kickoff Crew

In [39]:
# Kick off the crew and execute the process
result = crew.kickoff()

# Agent: Data Collection Specialist
## Task: Create an initial understanding of the project, its main features and the team working on it. Use the Project Tasks Fetcher Tool tool to gather basis tasks' data for an Asana project and the Task Data Fetcher Tool to gather task details.



# Agent: Data Collection Specialist
## Thought: I need to start by gathering the basic tasks' data from the Asana project using the Project Tasks Fetcher Tool to set a foundation for the report.
## Using tool: Asana Project Tasks Fetcher
## Tool Input: 
"{}"
## Tool Output: 
[]


# Agent: Data Collection Specialist
## Thought: Thought: It seems that there were no tasks fetched from the Asana project. I need to gather more information regarding specific tasks for further details. Since I have no task data, I cannot proceed with the Task Data Fetcher Tool. 
I will attempt to run the Project Tasks Fetcher Tool again to ensure there are no connection issues or try to gather any project identifiers that might 

## Usage Metrics and Costs

Let’s see how much it would cost each time if this crew runs at scale.

In [40]:
import pandas as pd

costs = 0.150 * (crew.usage_metrics.prompt_tokens + crew.usage_metrics.completion_tokens) / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([crew.usage_metrics.dict()])
df_usage_metrics

Total costs: $0.0008


,total_tokens,prompt_tokens,cached_prompt_tokens,completion_tokens,successful_requests
0,5547,4255,0,1292,7


## Report

In [41]:
from IPython.display import Markdown

markdown  = result.raw
Markdown(markdown)

# Sprint Report

## Sprint Overview
The current sprint has encountered significant challenges due to the inability to fetch tasks from the Asana project management tool. This situation has led to a lack of clarity regarding ongoing tasks and their status. Immediate action is required to address these issues to ensure that the project can continue moving forward effectively.

## Task Summary
- Number of tasks initiated: Unable to retrieve this data.
- Tasks completed: Unable to retrieve this data.
- Overdue tasks: Unable to retrieve this data.
- Tasks in progress: Unable to retrieve this data.

## Identified Issues and Blockers
1. **Data Retrieval Blocker**: The primary issue is the failure to access and retrieve data from Asana which severely limits visibility into ongoing tasks and their respective statuses.
2. **Impact on Reporting**: The absence of necessary data prevents the generation of reports that would inform stakeholders about progress, outstanding work, and any potential delays.

## Progress and Delays
- **Overall Progress**: Indeterminate due to lack of essential data points. This gap undermines our ability to make informed decisions regarding project direction.
- **Task Monitoring Delays**: The lack of data prevents tracking which tasks are falling behind schedule, potentially leading to significant delays in project timelines.
- **Resource Allocation Issues**: Inability to retrieve task information results in ineffective resource allocation, with potential underutilization of some team members and overloading of others.

## Team Performance Overview
Due to the lack of data in the system, it is challenging to evaluate team performance accurately. As a result, we risk misjudging individual contributions and overall team effectiveness during this sprint.

## Action Items and Recommendations
1. **Technical Troubleshooting**: Resolve the data retrieval issue from Asana as a priority to regain visibility into the project.
2. **Alternative Tracking**: Explore temporary alternatives for task tracking, possibly through manual tracking methods or other project management tools.
3. **Communication**: Establish open lines of communication among team members to manually document progress until the data issue is resolved.
4. **Review Project Management Tool**: If the issue persists, consider evaluating the feasibility of switching to a more reliable project management tool.
5. **Immediate Monitoring**: Set up interim weekly check-ins to monitor ongoing project progress based on team feedback.

## Conclusion
The project currently faces significant risks due to the data retrieval issue from Asana. Immediate attention is required to address these blockers to avoid prolonged delays and to enhance the overall project effectiveness. Active measures must be taken to ensure proper tracking and reporting as soon as possible.